# Naive Bayes
* Supervised algorithm
* Classification 
* Mostly used for NLP (Spam, Sentiment)
* Assume features are independent
* Gets effected when we have related features
* Doesn't require feature scaling  (Naive Bayes-No effect of missing values and feature scaling.ipynb)
* Naive bayes can handle missing values  (Naive Bayes-No effect of missing values and feature scaling.ipynb)

In [184]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for statistical data visualization
%matplotlib inline

In [185]:
df=pd.read_csv('F:/GitHub/Machine-Learning/Naive Bayes/adult.csv')

In [186]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [187]:
df.shape

(48842, 15)

In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [189]:
# At some places dataset has '?' we have to replace those with NaN, so that we can treat them as missing values
df.replace(to_replace='?', value=np.nan, inplace=True)

In [190]:
#freq of missing values
df.isnull().sum()

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

In [191]:
#replace missing values with mode
for cols in ['workclass', 'occupation', 'native-country']:
    df[cols].fillna(df[cols].mode()[0], inplace=True)

#again check freq of missing values
df.isnull().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

In [192]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,Private,103497,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,30,United-States,<=50K


In [193]:
#one hot encoding categorical column
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for col in ['workclass', 'education', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'marital-status']:
    le.fit(df[col])
    df[col]=le.transform(df[col])
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,3,226802,1,7,4,6,3,2,1,0,0,40,38,<=50K
1,38,3,89814,11,9,2,4,0,4,1,0,0,50,38,<=50K
2,28,1,336951,7,12,2,10,0,4,1,0,0,40,38,>50K
3,44,3,160323,15,10,2,6,0,2,1,7688,0,40,38,>50K
4,18,3,103497,15,10,4,9,3,4,0,0,0,30,38,<=50K


In [194]:
#Separate features and target
X=df.drop(['income'],axis=1)
y=df['income']


In [195]:
#Feature Scaling (Optional in case of Naive Bayes)
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

scaler.fit(X)
X=scaler.fit_transform(X)
# As .fit changes to numpy. Therefore changing back to dataframe
column_values=['age','workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
X=pd.DataFrame(X,columns=column_values)

X.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,-0.60,0.0,0.405170,-3.333333,-1.000000,1.0,-0.166667,0.666667,-2.0,0.0,0.0,0.0,0.0,0.0
1,0.05,0.0,-0.735527,0.000000,-0.333333,0.0,-0.500000,-0.333333,0.0,0.0,0.0,0.0,2.0,0.0
2,-0.45,-2.0,1.322379,-1.333333,0.666667,0.0,0.500000,-0.333333,0.0,0.0,0.0,0.0,0.0,0.0
3,0.35,0.0,-0.148399,1.333333,0.000000,0.0,-0.166667,-0.333333,-2.0,0.0,7688.0,0.0,0.0,0.0
4,-0.95,0.0,-0.621589,1.333333,0.000000,1.0,0.333333,0.666667,0.0,-1.0,0.0,0.0,-2.0,0.0


In [196]:
#separate test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
# check the shape of X_train and X_test
X_train.shape, X_test.shape

((34189, 14), (14653, 14))

In [197]:
#model trainig
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

#Testing
y_pred = gnb.predict(X_test)
y_pred

#confusion matrix
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_test, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[10566   572]
 [ 2363  1152]]
              precision    recall  f1-score   support

       <=50K       0.82      0.95      0.88     11138
        >50K       0.67      0.33      0.44      3515

    accuracy                           0.80     14653
   macro avg       0.74      0.64      0.66     14653
weighted avg       0.78      0.80      0.77     14653

